In [1]:
import numpy as np
import math

In [2]:
# softmax activation function
def softmax(val):
    return np.exp(val)/np.sum(np.exp(val))

In [3]:
# Leaky ReLU activation function
def relu(val):
    neg=[1 if i>0 else 0.01 for i in val[0]]
    val=val*neg
    return val

In [4]:
# i_layer: shape - (1,i)
# weights: shape - (i,j)
# return - output: shape - (1,j)
def forward_propogation(i_layer, weights,bias,func):
    output = np.matmul(i_layer,weights)+bias
    result=func(output)
#     print(output.shape)
    return  result, output

In [5]:
# hidden_val: shape - (1,j)
# value: shape - (1,k)
# target: shape - (1,k)
# return: shape - (j,k)
def output_weight_update(hidden_val,value,target):
    return np.matmul(hidden_val.T,(target-value))

In [6]:
# input_val: shape - (1,i)
# weight_h_o: shape - (j,k)
# threshold: shape - (1,j)
# value: shape - (1,k)
# target: shape - (1,k)
# return: shape - (i,j)
def hidden_weight_update(input_val,weight_h_o,value,target,threshold):
    val=np.matmul(target-value,weight_h_o.T)
    neg=[1 if i>0 else 0 for i in threshold[0]]
    val=val*neg
    return np.matmul(input_val.T,val)

In [7]:
def neural_net(data,label, hidden_layer,learning_rate,epochs,bias):
    ip_units=data.shape[1]
    output_layer=label.shape[1]
    
    # initialize weights for both layers
    weights_i_h=np.random.rand(ip_units,hidden_layer)
    biases_h=np.zeros((1,hidden_layer),dtype='float64')
#     biases_h=np.random.rand(1,hidden_layer)
    weights_h_o=np.random.rand(hidden_layer,output_layer)
    biases_o=np.zeros((1,output_layer),dtype='float64')
#     biases_o=np.random.rand(1,output_layer)

    for i in range(epochs):
        total_error=0.0
        hidden=[]
        for x,y in zip(data,label):

            # forward propogation
            input_val=x.reshape((1,len(x)))
            target_val=y.reshape((1,len(y)))
            hidden_activated_val,hidden_val =forward_propogation(input_val,weights_i_h,biases_h,relu)
            hidden.append(hidden_activated_val)
            output_val,_=forward_propogation(hidden_activated_val,weights_h_o,biases_o,softmax)

            #calculate total error
            total_error+= np.mean((output_val-target_val)**2)
            
            # calculate delta to update weights
            delta_h_o= output_weight_update(hidden_activated_val,output_val,target_val)
            delta_i_h= hidden_weight_update(input_val,weights_h_o,output_val,target_val,hidden_val)

            # weights update
            weights_h_o+=learning_rate*delta_h_o
            weights_i_h+=learning_rate*delta_i_h

        if i%int(epochs/10) is 0:
            print("Epoch-", str(i), ", Error - ", str(total_error))
    print()
    network={}
    network['weight']=[weights_i_h,weights_h_o]
    network['bias']=[biases_h,biases_o]
    print("Training Accuracy for wine dataset: ",evaluate(network,data,np.argmax(label,axis=1)+1))
    return network,hidden

In [8]:
def predict(network,data):
    
    predictions=[]
    
    for x in data:
#         print(x)
        input_val=x.reshape((1,len(x)))
        i=0
        func=relu
        for j,n in enumerate(zip(network['weight'],network['bias'])):
            if j==1:
                func=softmax
            input_val, _=forward_propogation(input_val,n[0],n[1],func)
#             print(i,"=",input_val)
            i+=1
        predictions.append(input_val)
    return predictions

In [9]:
def evaluate(model, x, y):
    output = predict(model, x)
    predictions=[j==np.argmax(i)+1 for i,j in zip(output,y)]
    return np.mean(predictions)

In [10]:
# Reads the dataset from a csv file
def read_csv(filename):
    with open(filename) as file:

        data=[line[:-1].split(',') for line in file.readlines()]
        data=np.array(data).astype('float64')
    return data

In [11]:
# Forms the output dataset to match the network output layer 
# where each output neuron corresponds to a particular class
def make_output_data(data):
    val={ 1:[1.0,0.0,0.0], 2:[0.0,1.0,0.0],3:[0.0,0.0,1.0]}
    label=[val[i] for i in data[:,0].astype('int')]
    return np.array(label)

### Multi-class Classifier Neural Network

A Multi-class classifier neural network that uses Cross-entropy loss, ReLU activation for hidden layer and Softmax activation for output layer.
Wine dataset used for classification

In [12]:
wine_train=read_csv('train_wine.csv')

In [13]:
wine_test=read_csv('test_wine.csv')

In [14]:
train_output=make_output_data(wine_train)

In [15]:
wine_train=wine_train[:,1:]

In [16]:
# Min-max normalization performed on the training and testing dataset
nmax=np.max(np.vstack((wine_train,wine_test[:,1:])),axis=0)
nmin=np.min(np.vstack((wine_train,wine_test[:,1:])),axis=0)

In [17]:
norm_wine_train=(wine_train - nmin)/(nmax-nmin)

In [18]:
norm_wine_test=(wine_test[:,1:]-nmin)/(nmax-nmin)

In [19]:
# Input Data
# Output data
# No. of neurons in the hidden layer
# Learning rate
# No. of epochs
# Boolean indicating if bias is added to the network
wine_network,wine_hidden=neural_net(norm_wine_train,train_output,4,0.01,20,False)

Epoch- 0 , Error -  35.56075546015582
Epoch- 2 , Error -  27.43573317943771
Epoch- 4 , Error -  19.471182337800176
Epoch- 6 , Error -  13.728868356983549
Epoch- 8 , Error -  10.0269075294662
Epoch- 10 , Error -  7.837526474298466
Epoch- 12 , Error -  6.527555183649636
Epoch- 14 , Error -  5.666061304688704
Epoch- 16 , Error -  5.044585453200642
Epoch- 18 , Error -  4.5662153627387205

Training Accuracy for wine dataset:  0.9602649006622517


In [20]:
# Get predictions on the test data
pred=predict(wine_network,norm_wine_test)

In [22]:
# Evaluate Training and testing accuracy for the network
print("Training Accuracy for wine dataset: ",evaluate(wine_network,norm_wine_train,np.argmax(train_output,axis=1)+1))
print("Testing Accuracy for wine dataset: ",evaluate(wine_network,norm_wine_test,wine_test[:,0]))

Training Accuracy for wine dataset:  0.9602649006622517
Testing Accuracy for wine dataset:  1.0
